# Hilos

Es una pieza de código, muy pequeña y compacta que puede ejecutar un sistema operativo. Los hilos de ejecución comparten recursos, como memoria, archivos abiertos, etc.

Un hilo es un tarea que puede ser "ejecutada al mismo tiempo" que otra tarea. Los hilos pueden tener diferentes estados: "Ejecutando", "Listo" y "bloqueado". Los hilos suelen estar dentro de los procesos.

El multi-threading es una técnica que permite desacoplar las tareas que no son secuencialmente dependientes.

Hay dos tipos de hilos:
 * en el espacio de kernel
 * en el espacio de usuario
 
## Ventajas del multithread
* puede correr más rapido en computadoras con múltiples CPUs
* Los hilos de un proceso comparten memoria. Y pueden compartir variables globales.


Para manejar hilos en Python tenemos el módulo *threading*


# Threading

En python existe el objeto Thread, que es una clase que representa un hilo de control. Se puede crear de dos maneras, pasando una función (callable object) o sobreescribiendo `run()`. Cuando se hereda de Thread solo se debe sobreeescribir `__init__(self)` y `run()`. 

Para que la ejecución del hilo comience se debe llamar al método `start()`, este invoca a `run()`. Se puede utilizar el método `is_alive()` para saber si un hilo está vivo o no.

El método `join()` bloquea la llamada al hilo que termina.

In [3]:
import time
from threading import Thread

def sleeper(i):
    print (f"thread {i} sleeps for 5 seconds")
    time.sleep(5)
    print (f"thread {i} woke up")

for i in range(10):
    t = Thread(target=sleeper, args=(i,))
    t.start()
    
t.join()

thread 0 sleeps for 5 seconds
thread 1 sleeps for 5 seconds
thread 2 sleeps for 5 secondsthread 3 sleeps for 5 seconds

thread 4 sleeps for 5 seconds
thread 5 sleeps for 5 seconds
thread 6 sleeps for 5 seconds
thread 7 sleeps for 5 seconds
thread 8 sleeps for 5 seconds
thread 9 sleeps for 5 seconds
thread 0 woke up
thread 3 woke up
thread 1 woke up
thread 4 woke up
thread 2 woke up
thread 7 woke upthread 5 woke upthread 6 woke up


thread 8 woke upthread 9 woke up



## Locks

Lock es un primitiva de sincronización que no pertenece a ningún hilo en particular cuando está cerrado. 

Un Lock está en uno de dos estados, `cerrado` o `abierto` (locked/unlocked). Se crea en estado abierto. Tiene dos métodos básicos, acquire() (adquirir) y release() (liberar). Cuando el estado es abierto, acquire() cambia el estado a cerrado y retorna inmediatamente. Cuando el estado es cerrado, acquire() bloquea hasta que una llamada a release() en otro hilo lo cambie a abierto, luego la llamada a acquire() lo restablece a cerrado y retorna. El método release() sólo debe ser llamado en el estado cerrado; cambia el estado a abierto y retorna inmediatamente. Si se realiza un intento de liberar un lock abierto, se lanzará un RuntimeError.



In [4]:
import threading
import time
import logging
import random

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-9s) %(message)s',)
                    
class Counter(object):
    def __init__(self, start = 0):
        self.lock = threading.Lock()
        self.value = start
    def increment(self):
        logging.debug('Waiting for a lock')
        self.lock.acquire()
        try:
            logging.debug('Acquired a lock')
            self.value = self.value + 1
        finally:
            logging.debug('Released a lock')
            self.lock.release()

def worker(c):
    for i in range(2):
        r = random.random()
        logging.debug('Sleeping %0.02f', r)
        time.sleep(r)
        c.increment()
    logging.debug('Done')

if __name__ == '__main__':
    counter = Counter()
    for i in range(2):
        t = threading.Thread(target=worker, args=(counter,))
        t.start()

    logging.debug('Waiting for worker threads')
    main_thread = threading.currentThread()
    for t in threading.enumerate():
        if t is not main_thread:
            t.join()
    logging.debug('Counter: %d', counter.value)

(Thread-17) Sleeping 0.18
(Thread-18) Sleeping 0.83
(MainThread) Waiting for worker threads
(Thread-17) Waiting for a lock
(Thread-17) Acquired a lock
(Thread-17) Released a lock
(Thread-17) Sleeping 0.08
(Thread-17) Waiting for a lock
(Thread-17) Acquired a lock
(Thread-17) Released a lock
(Thread-17) Done
(Thread-18) Waiting for a lock
(Thread-18) Acquired a lock
(Thread-18) Released a lock
(Thread-18) Sleeping 0.51
(Thread-18) Waiting for a lock
(Thread-18) Acquired a lock
(Thread-18) Released a lock
(Thread-18) Done


KeyboardInterrupt: 

## Queue

Las colas es otra manera de lograr sincronización entre procesos. Es muy util cuando los hilos necesitan intercambiar información de manera segura.

Más detalles: https://docs.python.org/es/3/library/queue.html#module-queue


In [5]:
import threading, queue

q = queue.Queue()

def worker():
    while True:
        item = q.get()
        print(f'Working on {item}')
        print(f'Finished {item}')
        q.task_done()
        
def worker2():
    # hay logica
    worker(dinero)
    

# turn-on the worker thread
threading.Thread(target=worker, daemon=True).start()

# send thirty task requests to the worker
for item in range(30):
    q.put({"id": 2, "otra": "hola"})
    
print('All task requests sent\n', end='')

# block until all tasks are done
q.join()
print('All work completed')


All task requests sent
Working on 0
Finished 0
Working on 1
Finished 1
Working on 2
Finished 2
Working on 3
Finished 3
Working on 4
Finished 4
Working on 5
Finished 5
Working on 6
Finished 6
Working on 7
Finished 7
Working on 8
Finished 8
Working on 9
Finished 9
Working on 10
Finished 10
Working on 11
Finished 11
Working on 12
Finished 12
Working on 13
Finished 13
Working on 14
Finished 14
Working on 15
Finished 15
Working on 16
Finished 16
Working on 17
Finished 17
Working on 18
Finished 18
Working on 19
Finished 19
Working on 20
Finished 20
Working on 21
Finished 21
Working on 22
Finished 22
Working on 23
Finished 23
Working on 24
Finished 24
Working on 25
Finished 25
Working on 26
Finished 26
Working on 27
Finished 27
Working on 28
Finished 28
Working on 29
Finished 29
All work completed


## Python GIL (Python Global Interpreter Lock)

Hace que en cualquier instante tiempo siempre existe uno y nada más que un hilo ejecutandose. Por lo que es imposible, hacer uso de multiple procesos con hilos. No todo es malo.

Que hace GIL:

* Limita las operaciones de hilos
* Ejecuciones paralelas están restringidas.
* Se asegura que un hilo se ejecute por vez.
* Simplifica tener que preocuparnos por detalles de memoria.

TODO:
 1. multiprocessing
 2. async 